In [3]:
!git clone https://github.com/NataliaVrabcova/assessment-1-neural-networks

Cloning into 'assessment-1-neural-networks'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 2.01 MiB | 11.41 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [5]:
import pandas as pd

# Adjust the path to match your repository structure
data = pd.read_csv('/content/assessment-1-neural-networks/healthcare_noshows_appointments.csv')

# Verify the dataset is loaded
print(data.head())

      PatientId  AppointmentID Gender ScheduledDay AppointmentDay  Age  \
0  2.987250e+13        5642903      F   2016-04-29     2016-04-29   62   
1  5.589978e+14        5642503      M   2016-04-29     2016-04-29   56   
2  4.262962e+12        5642549      F   2016-04-29     2016-04-29   62   
3  8.679512e+11        5642828      F   2016-04-29     2016-04-29    8   
4  8.841186e+12        5642494      F   2016-04-29     2016-04-29   56   

       Neighbourhood  Scholarship  Hipertension  Diabetes  Alcoholism  \
0    JARDIM DA PENHA        False          True     False       False   
1    JARDIM DA PENHA        False         False     False       False   
2      MATA DA PRAIA        False         False     False       False   
3  PONTAL DE CAMBURI        False         False     False       False   
4    JARDIM DA PENHA        False          True      True       False   

   Handcap  SMS_received  Showed_up  Date.diff  
0    False         False       True          0  
1    False        

In [6]:
# Importing libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam

Preprocessing the Data

In [7]:
# Selecting features and target variable
X = data.drop(['Showed_up', 'PatientId', 'AppointmentID'], axis=1)
y = data['Showed_up']

# Encoding categorical variables
X = pd.get_dummies(X, drop_first=True)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Defining the architecture of the neural network:

In [8]:
# Defining the neural network model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

Train the model with your preprocessed data:

In [9]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
2675/2675 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7769 - loss: 0.5436 - val_accuracy: 0.7979 - val_loss: 0.4712
Epoch 2/50
2675/2675 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7982 - loss: 0.4713 - val_accuracy: 0.7979 - val_loss: 0.4643
Epoch 3/50
2675/2675 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7969 - loss: 0.4609 - val_accuracy: 0.7979 - val_loss: 0.4550
Epoch 4/50
2675/2675 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7961 - loss: 0.4564 - val_accuracy: 0.7979 - val_loss: 0.4543
Epoch 5/50
2675/2675 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.7984 - loss: 0.4505 - val_accuracy: 0.7979 - val_loss: 0.4512
Epoch 6/50
2675/2675 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.7953 - loss: 0.4530 - val_accuracy: 0.7981 - val_loss: 0.4508
Epoch 7/50
2675/2675 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7973 - loss: 0.4512 - val_accuracy: 0.7982 - val_loss: 0.4516
Epoch 8/50
2675/2675 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.7996 - loss: 0.4457 -

Evaluate the model’s performance:

In [10]:
# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

669/669 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Accuracy: 0.7992
